In [7]:
from langchain_neo4j import Neo4jGraph
from langchain_ollama import ChatOllama
from langchain.prompts import PromptTemplate
from graph_parser_json import parse_stackup_to_graph

In [2]:
graph = Neo4jGraph(
    url="bolt://localhost:7687",
    username="neo4j",  # default username for Neo4j Desktop
    password="12345678",  # replace with your actual password
    refresh_schema=False
)

In [3]:
def clean_graph(graph):
    """Delete all nodes and relationships in the Neo4j database."""
    clean_query = "MATCH (n) DETACH DELETE n"
    graph.query(clean_query)
    print("Graph cleaned: All nodes and relationships deleted.")

In [4]:
# First clean the graph
clean_graph(graph)

Graph cleaned: All nodes and relationships deleted.


In [6]:
# Parse JSON and create graph
graph = parse_stackup_to_graph('single_layer_v1.json', graph)

Graph creation completed: Stackup structure imported into Neo4j


In [8]:
def generate_cypher_query(instruction, model_name="qwen2.5-coder:32b", graph=None):
    """Generate a Cypher query with schema awareness"""
    llm = ChatOllama(model=model_name)

    # Get schema information if graph is provided
    schema_info = ""
    if graph:
        # Get node labels
        labels_result = graph.query("CALL db.labels()")
        node_labels = [record["label"] for record in labels_result]

        # Get relationship types
        rel_result = graph.query("CALL db.relationshipTypes()")
        rel_types = [record["relationshipType"] for record in rel_result]

        schema_info = f"""
        Available node labels: {', '.join(node_labels)}
        Available relationship types: {', '.join(rel_types)}
        """

    prompt_template = f"""
    You are an expert in converting natural language instructions to Cypher queries for Neo4j.

    {schema_info}

    Given the following instruction, create a valid Cypher query using only the relationship types
    and node labels that exist in the database.

    Instruction: {{instruction}}

    Return only the Cypher query without any explanations or markdown formatting.
    """

    prompt = PromptTemplate(template=prompt_template, input_variables=["instruction"])
    chain = prompt | llm
    result = chain.invoke({"instruction": instruction})

    if hasattr(result, "content"):
        return result.content.strip()
    return str(result).strip()

In [9]:
def fomulate_response(graph_response, instruction, model_name="llama3.2:latest"):
    llm = ChatOllama(model=model_name)

    prompt_template = """
        Formulate a compact answer with short explanation to the following instruction based on the graph response.
        Graph response: {graph_response}
        Instruction: {instruction}
        """

    prompt = PromptTemplate(
        template=prompt_template,
        input_variables=["instruction", "graph_response"]
    )
    chain = prompt | llm
    result = chain.invoke({
        "instruction": instruction,
        "graph_response": str(graph_response)
    })

    if hasattr(result, "content"):
        return result.content.strip()
    return str(result).strip()

In [10]:
def safe_execute_query(graph, instruction, max_retries=2):
    """Execute a query with retry logic if it fails"""
    for attempt in range(max_retries + 1):
        cypher_query = generate_cypher_query(instruction, graph=graph)

        print(f"Attempt {attempt+1}: {cypher_query}")
        response = graph.query(cypher_query)

        if response:
            response = fomulate_response(response, instruction)

            return response, cypher_query

    return None, cypher_query


In [11]:
# Example: Convert a natural language instruction to a Cypher query
instruction = "Return the value of Layer2: thickness."

result, cypher_query = safe_execute_query(graph, instruction)
print(f"Successful query: {cypher_query}")
print("Query result:", result)

Attempt 1: MATCH (l:Layer {name: 'Layer2'})-[:THICKNESS]->(v:Variable)
RETURN v.value
Successful query: MATCH (l:Layer {name: 'Layer2'})-[:THICKNESS]->(v:Variable)
RETURN v.value
Query result: Based on the provided graph response, I can extract the necessary information.

The graph contains a single entry for Layer2 with a key-value pair of 'value' and '2.728mil'. Since there is only one layer in the graph, we can assume that it is the only thickness value available.

Therefore, the return statement would be:

`Return: 2.728 mil`
